# Battle of the Neighborhoods- Week 2

## Part 2: Web scraping of Population and demographics data of NYC

#### POPULATION DATA

Importing the required libraries

In [1]:
!pip install geopy
!pip install folium
import folium
import numpy as np
import pandas as pd
import requests
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim

from bs4 import BeautifulSoup

import csv

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries imported')

Libraries imported


Web scraping the data using Beautiful Soup

In [2]:
website_url = requests.get('https://en.wikipedia.org/wiki/Demographics_of_New_York_City').text
soup = BeautifulSoup(website_url,'lxml')
table = soup.find('table',{'class':'wikitable sortable'})
#print(soup.prettify())

headers = [header.text for header in table.find_all('th')]

table_rows = table.find_all('tr')        
rows = []
for row in table_rows:
   td = row.find_all('td')
   row = [row.text for row in td]
   rows.append(row)

with open('BON2_POPULATION1.csv', 'w') as f:
   writer = csv.writer(f)
   writer.writerow(headers)
   writer.writerows(row for row in rows if row)

In [3]:
Pop_data=pd.read_csv('BON2_POPULATION1.csv')
Pop_data.drop(Pop_data.columns[[7,8,9,10,11]], axis=1,inplace=True)
print('Data downloaded!')

Data downloaded!


In [4]:
Pop_data.columns = Pop_data.columns.str.replace(' ', '')
Pop_data.columns = Pop_data.columns.str.replace('\'','')
Pop_data.rename(columns={'Borough':'persons_sq_mi','County':'persons_sq_km'}, inplace=True)
Pop_data

,"NewYorkCitysfiveboroughs.mw-parser-output.navbar{display:inline;font-size:88%;font-weight:normal}.mw-parser-output.navbar-collapse{float:left;text-align:left}.mw-parser-output.navbar-boxtext{word-spacing:0}.mw-parser-output.navbarul{display:inline-block;white-space:nowrap;line-height:inherit}.mw-parser-output.navbar-brackets::before{margin-right:-0.125em;content:""[""}.mw-parser-output.navbar-brackets::after{margin-left:-0.125em;content:""]""}.mw-parser-output.navbarli{word-spacing:-0.125em}.mw-parser-output.navbar-miniabbr{font-variant:small-caps;border-bottom:none;text-decoration:none;cursor:inherit}.mw-parser-output.navbar-ct-full{font-size:114%;margin:07em}.mw-parser-output.navbar-ct-mini{font-size:114%;margin:04em}.mw-parser-output.infobox.navbar{font-size:100%}.mw-parser-output.navbox.navbar{display:block;font-size:100%}.mw-parser-output.navbox-title.navbar{float:left;text-align:left;margin-right:0.5em}vte\r\n",Jurisdiction\r\n,Population\r\n,GrossDomesticProduct\r\n,Landarea\r\n,Density\r\n,persons_sq_mi,squarekm,persons/mi2,persons/km2\r\n
0,The Bronx\r\n,\r\n Bronx\r\n,"1,418,207\r\n",42.695\r\n,"30,100\r\n",42.10\r\n,109.04\r\n,NaN,NaN,NaN
1,Brooklyn\r\n,\r\n Kings\r\n,"2,559,903\r\n",91.559\r\n,"35,800\r\n",70.82\r\n,183.42\r\n,NaN,NaN,NaN
2,Manhattan\r\n,\r\n New York\r\n,"1,628,706\r\n",600.244\r\n,"368,500\r\n",22.83\r\n,59.13\r\n,NaN,NaN,NaN
3,Queens\r\n,\r\n Queens\r\n,"2,253,858\r\n",93.310\r\n,"41,400\r\n",108.53\r\n,281.09\r\n,NaN,NaN,NaN
4,Staten Island\r\n,\r\n Richmond\r\n,"476,143\r\n",14.514\r\n,"30,500\r\n",58.37\r\n,151.18\r\n,NaN,NaN,NaN
5,City of New York,"8,336,817",842.343,"101,000",302.64,783.83,"27,547",NaN,NaN,NaN
6,State of New York,"19,453,561","1,731.910","89,000","47,126.40","122,056.82",412,NaN,NaN,NaN
7,Sources:[12][13][14] and see individual boroug...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
Pop_data.drop(columns= {'GrossDomesticProduct\r\n', 'squarekm', 'persons/mi2', 'persons/km2\r\n'}, inplace= True)
Pop_data

,"NewYorkCitysfiveboroughs.mw-parser-output.navbar{display:inline;font-size:88%;font-weight:normal}.mw-parser-output.navbar-collapse{float:left;text-align:left}.mw-parser-output.navbar-boxtext{word-spacing:0}.mw-parser-output.navbarul{display:inline-block;white-space:nowrap;line-height:inherit}.mw-parser-output.navbar-brackets::before{margin-right:-0.125em;content:""[""}.mw-parser-output.navbar-brackets::after{margin-left:-0.125em;content:""]""}.mw-parser-output.navbarli{word-spacing:-0.125em}.mw-parser-output.navbar-miniabbr{font-variant:small-caps;border-bottom:none;text-decoration:none;cursor:inherit}.mw-parser-output.navbar-ct-full{font-size:114%;margin:07em}.mw-parser-output.navbar-ct-mini{font-size:114%;margin:04em}.mw-parser-output.infobox.navbar{font-size:100%}.mw-parser-output.navbox.navbar{display:block;font-size:100%}.mw-parser-output.navbox-title.navbar{float:left;text-align:left;margin-right:0.5em}vte\r\n",Jurisdiction\r\n,Population\r\n,Landarea\r\n,Density\r\n,persons_sq_mi
0,The Bronx\r\n,\r\n Bronx\r\n,"1,418,207\r\n","30,100\r\n",42.10\r\n,109.04\r\n
1,Brooklyn\r\n,\r\n Kings\r\n,"2,559,903\r\n","35,800\r\n",70.82\r\n,183.42\r\n
2,Manhattan\r\n,\r\n New York\r\n,"1,628,706\r\n","368,500\r\n",22.83\r\n,59.13\r\n
3,Queens\r\n,\r\n Queens\r\n,"2,253,858\r\n","41,400\r\n",108.53\r\n,281.09\r\n
4,Staten Island\r\n,\r\n Richmond\r\n,"476,143\r\n","30,500\r\n",58.37\r\n,151.18\r\n
5,City of New York,"8,336,817",842.343,302.64,783.83,"27,547"
6,State of New York,"19,453,561","1,731.910","47,126.40","122,056.82",412
7,Sources:[12][13][14] and see individual boroug...,NaN,NaN,NaN,NaN,NaN


In [6]:
Pop_data.rename(columns = {'NewYorkCitysfiveboroughs.mw-parser-output.navbar{display:inline;font-size:88%;font-weight:normal}.mw-parser-output.navbar-collapse{float:left;text-align:left}.mw-parser-output.navbar-boxtext{word-spacing:0}.mw-parser-output.navbarul{display:inline-block;white-space:nowrap;line-height:inherit}.mw-parser-output.navbar-brackets::before{margin-right:-0.125em;content:"["}.mw-parser-output.navbar-brackets::after{margin-left:-0.125em;content:"]"}.mw-parser-output.navbarli{word-spacing:-0.125em}.mw-parser-output.navbar-miniabbr{font-variant:small-caps;border-bottom:none;text-decoration:none;cursor:inherit}.mw-parser-output.navbar-ct-full{font-size:114%;margin:07em}.mw-parser-output.navbar-ct-mini{font-size:114%;margin:04em}.mw-parser-output.infobox.navbar{font-size:100%}.mw-parser-output.navbox.navbar{display:block;font-size:100%}.mw-parser-output.navbox-title.navbar{float:left;text-align:left;margin-right:0.5em}vte\r\n' : 'Borough',
                   'Jurisdiction\r\n':'County',
                   'Population\r\n':'Estimate_2017', 
                   'Landarea\r\n':'square_miles',
                    'Density\r\n':'square_km'}, inplace=True)
Pop_data

,Borough,County,Estimate_2017,square_miles,square_km,persons_sq_mi
0,The Bronx\r\n,\r\n Bronx\r\n,"1,418,207\r\n","30,100\r\n",42.10\r\n,109.04\r\n
1,Brooklyn\r\n,\r\n Kings\r\n,"2,559,903\r\n","35,800\r\n",70.82\r\n,183.42\r\n
2,Manhattan\r\n,\r\n New York\r\n,"1,628,706\r\n","368,500\r\n",22.83\r\n,59.13\r\n
3,Queens\r\n,\r\n Queens\r\n,"2,253,858\r\n","41,400\r\n",108.53\r\n,281.09\r\n
4,Staten Island\r\n,\r\n Richmond\r\n,"476,143\r\n","30,500\r\n",58.37\r\n,151.18\r\n
5,City of New York,"8,336,817",842.343,302.64,783.83,"27,547"
6,State of New York,"19,453,561","1,731.910","47,126.40","122,056.82",412
7,Sources:[12][13][14] and see individual boroug...,NaN,NaN,NaN,NaN,NaN


Shifting data in the last two rows

In [7]:
Pop_data.loc[5:,['square_km','persons_sq_mi']] = Pop_data.loc[2:,['square_km','persons_sq_mi']].shift(1,axis=1)
Pop_data.loc[5:,['square_miles','square_km']] = Pop_data.loc[2:,['square_miles','square_km']].shift(1,axis=1)
Pop_data.loc[5:,['Estimate_2017','square_miles']] = Pop_data.loc[2:,['Estimate_2017','square_miles']].shift(1,axis=1)
Pop_data.loc[5:,['County','Estimate_2017']] = Pop_data.loc[2:,['County','Estimate_2017']].shift(1,axis=1)
Pop_data.loc[5:,['Borough','County']] = Pop_data.loc[2:,['Borough','County']].shift(1,axis=1)
Pop_data

,Borough,County,Estimate_2017,square_miles,square_km,persons_sq_mi
0,The Bronx\r\n,\r\n Bronx\r\n,"1,418,207\r\n","30,100\r\n",42.10\r\n,109.04\r\n
1,Brooklyn\r\n,\r\n Kings\r\n,"2,559,903\r\n","35,800\r\n",70.82\r\n,183.42\r\n
2,Manhattan\r\n,\r\n New York\r\n,"1,628,706\r\n","368,500\r\n",22.83\r\n,59.13\r\n
3,Queens\r\n,\r\n Queens\r\n,"2,253,858\r\n","41,400\r\n",108.53\r\n,281.09\r\n
4,Staten Island\r\n,\r\n Richmond\r\n,"476,143\r\n","30,500\r\n",58.37\r\n,151.18\r\n
5,NaN,City of New York,"8,336,817",842.343,302.64,783.83
6,NaN,State of New York,"19,453,561","1,731.910","47,126.40","122,056.82"
7,NaN,Sources:[12][13][14] and see individual boroug...,NaN,NaN,NaN,NaN


In [8]:
Pop_data = Pop_data.fillna('')
Pop_data

,Borough,County,Estimate_2017,square_miles,square_km,persons_sq_mi
0,The Bronx\r\n,\r\n Bronx\r\n,"1,418,207\r\n","30,100\r\n",42.10\r\n,109.04\r\n
1,Brooklyn\r\n,\r\n Kings\r\n,"2,559,903\r\n","35,800\r\n",70.82\r\n,183.42\r\n
2,Manhattan\r\n,\r\n New York\r\n,"1,628,706\r\n","368,500\r\n",22.83\r\n,59.13\r\n
3,Queens\r\n,\r\n Queens\r\n,"2,253,858\r\n","41,400\r\n",108.53\r\n,281.09\r\n
4,Staten Island\r\n,\r\n Richmond\r\n,"476,143\r\n","30,500\r\n",58.37\r\n,151.18\r\n
5,,City of New York,"8,336,817",842.343,302.64,783.83
6,,State of New York,"19,453,561","1,731.910","47,126.40","122,056.82"
7,,Sources:[12][13][14] and see individual boroug...,,,,


In [15]:
Pop_data['Borough']=Pop_data['Borough'].replace(to_replace='\r\n', value='', regex=True)
Pop_data['County']=Pop_data['County'].replace(to_replace='\r\n', value='', regex=True)
Pop_data['Estimate_2017']=Pop_data['Estimate_2017'].replace(to_replace='\r\n', value='', regex=True)
Pop_data['square_miles']=Pop_data['square_miles'].replace(to_replace='\r\n', value='', regex=True)
Pop_data['square_km']=Pop_data['square_km'].replace(to_replace='\r\n', value='', regex=True)
Pop_data['persons_sq_mi']=Pop_data['persons_sq_mi'].replace(to_replace='\r\n', value='', regex=True)
Pop_data

,Borough,County,Estimate_2017,square_miles,square_km,persons_sq_mi
0,The Bronx,Bronx,"1,418,207","30,100",42.10,109.04
1,Brooklyn,Kings,"2,559,903","35,800",70.82,183.42
2,Manhattan,New York,"1,628,706","368,500",22.83,59.13
3,Queens,Queens,"2,253,858","41,400",108.53,281.09
4,Staten Island,Richmond,"476,143","30,500",58.37,151.18
5,,City of New York,"8,336,817",842.343,302.64,783.83
6,,State of New York,"19,453,561","1,731.910","47,126.40","122,056.82"
7,,Sources:[12][13][14] and see individual boroug...,,,,


In [16]:
Pop_data.drop(Pop_data.index[7])

,Borough,County,Estimate_2017,square_miles,square_km,persons_sq_mi
0,The Bronx,Bronx,"1,418,207","30,100",42.10,109.04
1,Brooklyn,Kings,"2,559,903","35,800",70.82,183.42
2,Manhattan,New York,"1,628,706","368,500",22.83,59.13
3,Queens,Queens,"2,253,858","41,400",108.53,281.09
4,Staten Island,Richmond,"476,143","30,500",58.37,151.18
5,,City of New York,"8,336,817",842.343,302.64,783.83
6,,State of New York,"19,453,561","1,731.910","47,126.40","122,056.82"


In [17]:
Pop_data.to_csv('BON2_POPULATION.csv',index=False)

#### DEMOGRAPHICS DATA

In [18]:
demo_data= pd.read_excel(r'C:\Users\nruta\Desktop\NYC_demo.xlsx')
demo_data

,Racial Composition,2010,1990,1970,1940
0,White,0.440,0.523,0.766,0.936
1,Non-Hispanic,0.333,0.432,0.629,0.92
2,Black or African American,0.255,0.287,0.211,0.061
3,Hispanic or Latino,0.286,0.244,0.162,0.016
4,Asian,0.127,0.070,0.012,--


In [21]:
demo_data.to_csv('BON2_DEMOGRAPHICS.csv',index=False)